In [33]:
import tensorflow as tf

class Model:
    def __init__(self):
        self.sess = None
        self.X = None
        self.y = None
        self.layer_keeps = None
        self.vars = None
        self.keep_prob_train = None
        self.keep_prob_test = None

    def run(self, fetches, X=None, y=None, mode='train'):
            feed_dict = {}
            if type(self.X) is list:
                for i in range(len(X)):
                    feed_dict[self.X[i]] = X[i]
            else:
                feed_dict[self.X] = X
            if y is not None:
                feed_dict[self.y] = y
            if self.layer_keeps is not None:
                if mode == 'train':
                    feed_dict[self.layer_keeps] = self.keep_prob_train
                elif mode == 'test':
                    feed_dict[self.layer_keeps] = self.keep_prob_test
            return self.sess.run(fetches, feed_dict)

    def dump(self, model_path):
        var_map = {}
        for name, var in self.vars.iteritems():
            var_map[name] = self.run(var)
        pkl.dump(var_map, open(model_path, 'wb'))
        print('model dumped at', model_path)

In [36]:
class FNN(Model):
    def __init__(self, field_sizes=None, num_statistic_dim=0, embed_size=10, layer_sizes=None, layer_acts=None, drop_out=None,
                 embed_l2=None, layer_l2=None, init_path=None, opt_algo='gd', learning_rate=1e-2, random_seed=None):
        Model.__init__(self)
        init_vars = []
        # 这里是域的个数，也就是54个
        num_inputs = len(field_sizes)
        # xavier是个矩阵，长是域中可能值的个数，比如说custommomoid就是几千个广告主，宽是隐向量的维度，这里是10，所以是54个矩阵
        # 占了 54 * 30000 * 10的内存
        for i in range(num_inputs):
            init_vars.append(('embed_%d' % i, [field_sizes[i], embed_size], 'xavier', dtype))
        node_in = num_inputs * embed_size
        for i in range(len(layer_sizes)):
            # layer size目前是3层，每层加偏置项，1000那一层，w就是embeding后的个数*1000，下一层是1000*100
            init_vars.append(('w%d' % i, [node_in, layer_sizes[i]], 'xavier', dtype))
            # 每一个节点都有偏置项，其实可以化简为只有一个
            init_vars.append(('b%d' % i, [layer_sizes[i]], 'zero', dtype))
            node_in = layer_sizes[i]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = [tf.sparse_placeholder(dtype) for i in range(num_inputs)]
            self.y = tf.placeholder(dtype)
            self.keep_prob_train = 1 - np.array(drop_out)
            self.keep_prob_test = np.ones_like(drop_out)
            self.layer_keeps = tf.placeholder(dtype)
            self.vars = utils.init_var_map(init_vars, init_path)
            w0 = [self.vars['embed_%d' % i] for i in range(num_inputs)]
            # 这一句进行embeding，每一个域中有一个不是一的值，刮取出隐向量，最后concat之后就是54*10=540维，这时已经不是0和1
            # ftrl与adagrad主要是在这一层网络要用，因为各种id的频率很不一致
            # 如果要把离散值加入，就要变成xw.extend(数值型特征)
            xw = tf.concat([tf.sparse_tensor_dense_matmul(self.X[i], w0[i]) for i in range(num_inputs)], 1)
            # x:[[0,1,0],[1,0,0],1.2,1.3,0.9]
            xw2 = tf.concat([xw, [self.X[i] for i in range(num_inputs, num_inputs + num_statistic_dim)]], 1)
            l = xw2

            for i in range(len(layer_sizes)):
                wi = self.vars['w%d' % i]
                bi = self.vars['b%d' % i]
                print(l.shape, wi.shape, bi.shape)
                l = tf.nn.dropout(
                    utils.activate(
                        tf.matmul(l, wi) + bi,
                        layer_acts[i]),
                    self.layer_keeps[i])

            l = tf.squeeze(l)
            self.y_prob = tf.sigmoid(l)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=self.y))
            if layer_l2 is not None:
                self.loss += embed_l2 * tf.nn.l2_loss(xw)
                for i in range(len(layer_sizes)):
                    wi = self.vars['w%d' % i]
                    self.loss += layer_l2[i] * tf.nn.l2_loss(wi)
            self.optimizer = utils.get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

In [25]:
num_inputs = 2
X = [tf.sparse_placeholder(tf.float32) for i in range(num_inputs)]
X.append(tf.placeholder(tf.float32,[1]))

X

 <tf.Tensor 'Placeholder_179:0' shape=(1,) dtype=float32>]

In [31]:
w00 = tf.Variable(tf.random_uniform([3000, 10], minval=-1e-3, maxval=1e-3, dtype=tf.float32),
                                            name='xavier', dtype=tf.float32)
w01 = tf.Variable(tf.random_uniform([3000, 10], minval=-1e-3, maxval=1e-3, dtype=tf.float32),
                                            name='xavier', dtype=tf.float32)

w0 = [w00, w01]
xw = tf.concat([tf.sparse_tensor_dense_matmul(X[i], w0[i]) 
                for i in range(num_inputs)].append([X[2]]), 1)
#xw = tf.concat(wx, X[2])

#xw2 = tf.concat([X,X,X], 1)



xw

ValueError: Tried to convert 'input' to a tensor and failed. Error: None values not supported.

In [ ]:
tf.reshape([0,0],shape=[-1,0])